In [135]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, matthews_corrcoef, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData
import DB_reader

import importlib

In [136]:
importlib.reload(ML)

def teach_models(X_train, y_train):
    models=[]

    X_train, y_train = SMOTETomek().fit_resample(X_train, y_train)

    models.append(ML.NeuralNetworks(X_train, y_train))
    models.append(ML.KNN(X_train, y_train))
    models.append(ML.DecisionTree(X_train, y_train))
    models.append(ML.LinearRegressionModel(X_train, y_train))
    models.append(ML.LogisticRegressionModel(X_train, y_train))
    models.append(ML.RandomForest(X_train, y_train))
    models.append(ML.SupportVectorMachines(X_train, y_train))
    models.append(ML.SupportVectorRegression(X_train, y_train))
    models.append(ML.TreeGradientBoosting(X_train, y_train))

    return models

In [137]:
def predict_tomorrow_weather(models):
    results=[]
    today_data=DB_reader.getWeatherForecast()
    today_data=PrepareData.standarize_row(today_data, df.iloc[:,1:].columns.values)

    if type(models)==list:
        for model in models:
            results.append(model.predict(today_data)>0.5)
        result=(sum(results)/len(results))>0.5

    else:
        result=models.predict(today_data)>0.5

    answer="Tomorrow will be rainy" if result else "Tomorrow will be sunny"
    return answer

In [138]:
def score_models(models, X_test, y_test):
    results=[]
    for model in models:
        scores={}
        y_pred=model.predict(X_test)>0.5
        scores['Model']=model.__str__()
        scores['Accuracy']=accuracy_score(y_test, y_pred)
        scores['F1_score']=f1_score(y_test, y_pred)
        scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred)
        scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred)
        scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred))
        scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred)
        scores['Confusion_matrix']=confusion_matrix(y_test, y_pred)

        results.append(scores)
    return results

In [139]:
def score_merged_models(models, X_test, y_test):
    merged_scores={}
    results=pd.DataFrame()
    num=1
    for model in models:
        column="Model_"+str(num)
        results[column]=list(model.predict(X_test)>0.5)
        num+=1

    results=results.astype(np.int32)

    results["Sum"]=results.sum(axis=1)/len(models)>0.5

    y_pred_merged=results["Sum"].values>0.5

    merged_scores['Model']='Merged model'
    merged_scores['Accuracy']=accuracy_score(y_test, y_pred_merged)
    merged_scores['F1_score']=f1_score(y_test, y_pred_merged)
    merged_scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred_merged)
    merged_scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred_merged)
    merged_scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred_merged))
    merged_scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred_merged)
    merged_scores['Confusion_matrix']=confusion_matrix(y_test, y_pred_merged)

    return merged_scores

In [140]:
importlib.reload(PrepareData)

df=PrepareData.get_data()

df=df.dropna()

In [141]:
y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

models=teach_models(X_train, y_train)

Epoch 1/25
329/329 [==============================] - 2s 4ms/step - loss: 0.6908 - f1: 0.4175 - accuracy: 0.5143 - val_loss: 0.6775 - val_f1: 0.6920 - val_accuracy: 0.6712
Epoch 2/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6792 - f1: 0.4932 - accuracy: 0.5691 - val_loss: 0.6664 - val_f1: 0.3494 - val_accuracy: 0.5771
Epoch 3/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6626 - f1: 0.4916 - accuracy: 0.5961 - val_loss: 0.6255 - val_f1: 0.7075 - val_accuracy: 0.6849
Epoch 4/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6552 - f1: 0.5194 - accuracy: 0.6035 - val_loss: 0.6201 - val_f1: 0.7105 - val_accuracy: 0.6872
Epoch 5/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6430 - f1: 0.5497 - accuracy: 0.6183 - val_loss: 0.6109 - val_f1: 0.6854 - val_accuracy: 0.6861
Epoch 6/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6492 - f1: 0.5646 - accuracy: 0.5982 - val_loss: 0.6208 - val_f1

In [142]:
result=predict_tomorrow_weather(models)
print(result)

1/1 [==============================] - 0s 127ms/step
Tomorrow will be rainy


In [145]:
scores=score_models(models,X_test,y_test)
scores[1]

63/63 [==============================] - 0s 2ms/step


{'Model': 'KNeighborsClassifier()',
 'Accuracy': 0.6689895470383276,
 'F1_score': 0.7107438016528925,
 'Matthwes_correlation_coefficient_(MCC)': 0.3355822026540532,
 'Mean_squared_error_(MSE)': 0.3310104529616725,
 'Square_root_of_mean_squared_error_(RMSE)': 0.575335078855507,
 'Mean_absolute_error_(MAE)': 0.3310104529616725,
 'Confusion_matrix': array([[527, 240],
        [425, 817]], dtype=int64)}

In [144]:
# merged_scores=score_merged_models(models,X_test,y_test)
best_score=0
best_models=[]
num=0

import itertools
for L in range(2,len(models) + 1):
    for subset in itertools.combinations(models, L):
        merged_scores=score_merged_models(list(subset),X_test,y_test)
        if merged_scores['F1_score']>best_score:
            best_score=merged_scores['F1_score']
            best_models=list(subset)
        if num%100==0:
            print(num)
        num+=1

print(best_score)
print(best_models)

63/63 [==============================] - 0s 2ms/step
0
63/63 [==============================] - 0s 2ms/step
100
63/63 [==============================] - 0s 2ms/step
200
63/63 [==============================] - 0s 2ms/step
300
63/63 [==============================] - 0s 2ms/step
400
63/63 [==============================] - 0s 2ms/step
500
63/63 [==============================] - 0s 2ms/step
0.7661855670103093
[KNeighborsClassifier(), RandomForestClassifier(max_depth=3), LGBMClassifier()]
